Am facut mai multe teste ca sa vad care configurare are cele mai bune rezultate

Unele lucruri sunt adevarate pentru toate versiunile folosite:
1. Tipul de eclipsa este encoded in 0 daca e Totala, sau 1 daca e Anuala
2. Latitudinea si Longitudinea sunt convertite intr-un range de -180, 180.
3. Timpul este convertid in secunde (din XmYs)

Scopul final este de a folosi XGBoost. Calcularea distantei intre 2 puncte pe latitudine si longitudine este definita mai jos, inainte de antrenarea modelului. 
Am incercat diferite configurari si combinatii de coloane, insa cea mai buna versiune mi-a dat doar:   

   **distanca medie = 8845.415935927434 km**   
   
   **distanta mediana = 9010.216275333436 km** 
   
   **distanta minimă = 910.6896836119255 km**

Aceste rezultate au fost dupa antrenarea unui subset de 80% din date, si testarea unui subset de 20% din date. Dar, deoarece vrem sa antrenam toate datele (inclusiv finale), cat mai bine, pentru a prezice in viitor, scopul final este de a antrena toate datele. Daca antrenam pe tot setul de date, si testam pe acele date, avem aceste rezultate:

   **distanca medie =  6302.5127052297075 km**   
   
   **distanta mediana = 6805.434560878371 km** 
   
   **distanta minimă = 51.352887716522886 km**

In [ ]:
# Instalam XGBoost in linia de comanda
!pip show xgboost

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor,MultiOutputClassifier
import xgboost as xgb
import math

# Preprocesare

### Functii pentru preprocesarea datelor
In celula de mai jos avem functii pentru a preprocesa datele inainte de antrenare. Fiecare functie are o descriere

In [ ]:
def drop_columns(dataframe,to_keep):
    """
    Sterge toate coloanele care nu sunt in lista din parametrul 2, si returneaza
    dataframe-ul nou cu indexul resetat
    """
    dataframe = dataframe.drop(columns=[col for col in df if col not in to_keep])
    dataframe.reset_index(inplace=True) 
    dataframe.drop(["index"],axis=1,inplace=True)
    return dataframe

def convertTime(x):
    """
    Converteste timpul din XmYs, in secunde
    Daca X nu are formatul corect, returneaza 0
    """
    if type(x) == type(float()) or x == "-":
        return 0
    
    x = x.replace("m"," ")
    x = x.replace("s","")
    x = x.split(" ")
    
    seconds = int(x[1])+int(x[0])*60
    return seconds

def encodeEclipses(x):
    """
    Functia da Encode la 
    Daca eclipsa este T, este eclipsa Totala
    Daca eclipsa este A, este eclipsa anuala
    """
    if x == "T":
        return 0
    elif x == "A":
        return 1
    
    
    
def convertLat(x):
    """
    Converteste latitudinea dintr-un range de [180S, 180N] in [-180, 180]
    """
    c = x[-1]
    
    x = float(x[:-1])
    if c == "S" or c == "s":
        x = x*-1

    return x
def convertLong(x):
    """        
    Converteste lognitudinea dintr-un range de [180W, 180E] in [-180, 180]        
    """
    c= x[-1]
    x = float(x[:-1])
    if c == "W" or c == "w":
        x = x*-1
    return x

### Citirea si preprocesarea datelor

In [ ]:
# Citim datasetul
df = pd.read_csv("../input/solar-eclipses/solar.csv")
df.head()

In [ ]:
# Putem vedea toate tipurile de eclipse. Dintre acestea, cele care sunt Hibride sau Partiale nu ne intereseaza
print("Every type:",df['Eclipse Type'].unique())

types_drop = []
for x in df['Eclipse Type'].unique():
    if x[0] == "P" or x[0] == "H":
        types_drop.append(x)


print("Types to drop:",types_drop)

In [ ]:

print("Every type:",df['Eclipse Type'].unique())
# O sa avem un numar de len(types_drop) operatii, in care dam drop
for tip in types_drop:
    df.drop(df[df['Eclipse Type'] == tip].index, inplace=True)    
    
print("Types after drop:",df['Eclipse Type'].unique())


In [ ]:
# De asemenea, trebuie sa mai stergem randurile care nu ni le trebuie. Astfel, avem o lista care contine toate coloanele pe care vrem sa le pastram
to_keep = ['Saros Number', 'Eclipse Type', 'Gamma','Eclipse Magnitude','Latitude','Longitude','Sun Altitude','Sun Azimuth','Path Width (km)','Central Duration']
df = drop_columns(df,to_keep)

df.head()

In [ ]:
# Pentru ca am facut stergeri, trebuie sa dam reset la index
df.reset_index(inplace=True) 
df.drop(["index"],axis=1,inplace=True)

In [ ]:
# Avem unele valori in coloana `Path Width (km)` care sunt '-', iar pe acelea le stergem

print("Coloane inainte de stergeri:",len(df))
indexes_to_drop = []
for i in range(len(df)):
    
    isInt = 0
    # We check if it can be transformed in an int
    try: 
        int(df.iloc[i]["Path Width (km)"])
        isInt = 1
    except ValueError:
        isInt = 0
    if isInt == 0:    
        indexes_to_drop.append(i)
        
df.drop(indexes_to_drop,inplace=True)

df.reset_index(inplace=True) 
df.drop(["index"],axis=1,inplace=True)

print("Coloane dupa stergeri:",len(df))
# Putem vedea mai jos ca am pierdut 181 de randuri, dar este necesar.

In [ ]:
# Restul de preprocesari
df['Central Duration'] = df['Central Duration'].apply(lambda x:convertTime(x))
df['Eclipse Type'] = df['Eclipse Type'].apply(lambda x:encodeEclipses(x))
df['Latitude'] = df['Latitude'].apply(lambda x : convertLat(x))
df['Longitude'] = df['Longitude'].apply(lambda x : convertLong(x))

In [ ]:
# Asa arata setul de date acum
df.head()

Cream o coloana noua care sa se numeasca Target Latitude (TargetLat) si Target Longitude (TargetLong), pe care trebuie sa prezica modelul 

In [ ]:
# targetLat[i] si targetLong[i] sunt latitudinea[i+1] si longitudinea[i+1], adica cele de pe randul urmator
targetLat = []
targetLong = []

for i,row in df.iterrows():
    if i < len(df)-1:

        # Le încărcăm în acestă ordine: Latitute, Longitute, TargetLatitute, TargetLongitute   
        targetLat.append(df.loc[i+1].Latitude)
        targetLong.append(df.loc[i+1].Longitude)

        
# Vom sterge ultimul rand pentru ca nu avem targetLat si targetLong pentru el
df.drop(df.tail(1).index,inplace=True) # drop last n rows
df["TargetLat"] = targetLat
df["TargetLong"] = targetLong
df.head()

In [ ]:
print("Numarul total de randuri:",len(df))

In [ ]:
# Dintre toate coloanele de mai sus, pastram doar acestea
to_keep = ['Latitude','Longitude','Sun Azimuth','Central Duration','TargetLat','TargetLong']
df = drop_columns(df,to_keep)

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True) 
df.drop(["index"],axis=1,inplace=True)

In [ ]:
print("Numarul total de randuri:",len(df))

In [ ]:
# Deoarece partea de predictie in viitor se bazeaza pe prezicere continua pe datele care au fost prezise, nu putem avea decat coloanele de latitudine si longitudine
to_keep = ['Latitude','Longitude','TargetLat','TargetLong']
df = drop_columns(df,to_keep)

In [ ]:
df.head()

# Antrenarea modelului

In [ ]:
# Trebuie sa despartim datele in X si y. X sunt datele de input pentru antrenare, iar y sunt datele pe care trebuie sa le prezica modelul

X = []
y = []
for vals in df.values:
    
    # Pastram in y doar ultimele 2, iar restul in X
    X.append(vals[:-2])
    y.append([vals[-2],vals[-1]])
    

X = np.asarray(X)
y = np.asarray(y)


# Impartim datele pe care le avem in training si testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, shuffle=False)

In [ ]:
# Definim o functie pentru a calcula distanta dintre 2 puncte latiudinale si longitudinale.
# Aceasta functie o vom folosi pentru a calcula acuratetea
def calculateDistances(latTrue,lonTrue,latPred,lonPred):
    R = 6373.0


    lat1 = math.radians(latTrue)
    lon1 = math.radians(lonTrue)
    lat2 = math.radians(latPred)
    lon2 = math.radians(lonPred)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c    
    
    return distance

In [ ]:
# Definim modelul pentru a-l antrena
XGB_Loc_Regressor =  MultiOutputRegressor(xgb.XGBRegressor(learning_rate = 0.01, max_depth = 23, objective = "reg:squarederror"))
XGB_Loc_Regressor.fit(X, y,verbose=2)

In [ ]:
def get_results(predictions,y_true):
    # Calculam media si mediana a rezultatelor obtinute
    # Rezultatele sunt distanta de la locatia prezisa, la locatia reala
    k = 0
    distances = []
    for i in range(len(predictions)):

        dist = calculateDistances(y_true[i][0],y_true[i][1],predictions[i][0],predictions[i][1])
        if dist <= 500:
            k += 1
        distances.append(dist)
    distances.sort()
    
    return distances

In [ ]:
# Prezicem cu modelul antrenat
pred = XGB_Loc_Regressor.predict(X)

# Apelam functia pentru a lua rezultatele
distances = get_results(pred,y)

print("Mean distances =",np.mean(distances))
print("Average distances =",np.median(distances))
print("Minim distances =",min(distances))

In [ ]:
# Aici calculam sa vedem cat la % din distance sunt sub 3000km
area_covered = 3000

under = 0
total = 0
for i in range(len(distances)):
    if distances[i]<=area_covered:
        under+=1        
    total += 1
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Total areas under {}kms: {}".format(area_covered,under))
print("Total number of areas: {}".format(total))
print("Percentage: {:.4f}%".format((under*100)/total))

In [ ]:
# Dam plot la distance 
plt.hist(distances)
plt.xlabel('Distances (km)')
plt.show()

In [ ]:
# Plot la importanta featurilor in model
plt.title("Feature importance for latitude")
plt.bar(range(len(XGB_Loc_Regressor.estimators_[0].feature_importances_)), XGB_Loc_Regressor.estimators_[0].feature_importances_)
plt.plot()

In [ ]:
plt.title("Feature importance for longitude")
plt.bar(range(len(XGB_Loc_Regressor.estimators_[1].feature_importances_)), XGB_Loc_Regressor.estimators_[1].feature_importances_)
plt.plot()

# Predictie

In [ ]:
# Pentru predictie vom crea o functie care sa inceapa de la ultima valoare din X, sa prezica acea valoare, si dupa sa
# prezica pentru acea valoare de asemenea.

N = 10 # Cate preziceri in viitor sa se faca

predictions = []

# Cream o variabila noua in care sa se tina featurile pentru predictie
X_future = [] # Ultima coloana din X
X_future.append([X[len(X)-1][0],X[len(X)-1][1]])
print(X_future)
for i in range(N):
    
    pred = XGB_Loc_Regressor.predict([X_future[i]])
    
    # Salvam ce am prezis in predictions, dar si in X_Future pentru a prezice urmatorul
    X_future.append(pred[0])
    predictions.append([pred[0][0],pred[0][1]])
print(predictions)
